In [1]:
import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from api_key_file import api_key
from IPython.display import JSON
import requests
import json
import pandas as pd
import csv

api_service_name = "youtube"
api_version = "v3"

In [2]:
channel_ids = ["UCLKZ20yD2tNMBOkSDZo4FeQ",
"UCzAF54cHk1ZO82af-8E3qOQ",
"UCteRPiisgIoHtMgqHegpWAQ",
"UCw_LFe2pS8x3NyipGNJgeEA",
"UCLLw7jmFsvfIVaUFsLs8mlQ",
"UCDybamfye5An6p-j1t2YMsg",
"UC7cs8q-gJRlGwj4A8OmCmXg",
"UCQiNyL7ik4FIlV2UCvojq0g",
"UCJQJAI7IjbLcpsjWdSzYz0Q",
"UCu1xbgCV5o48h_BYCQD7KJg",
"UCggXQcy57bwxUeZI1LB8DCw",
"UCPcyffOFc7YGyDgqmvZJs7Q",
"UCC8nkE-p-7S0ZPJXrEs2QNA",
"UCcJQ96WlEhJ0Ve0SLmU310Q",
"UCXqKK0ZZO5EhS6USGkGcuiw",
"UCLpovxJVLBZrXJGCymB6LYw",
"UC2fYCeDDpXrQVjSPdJY4lyA",
"UC4MZ7zUHb5eAxU75Dc_nqdQ",
"UC8butISFwT-Wl7EV0hUK0BQ",
"UCsvqVGtbbyHaMoevxPAq9Fg",
"UCilJYqsETFZ7NG9g1xVKfcQ",
"UCsLo154Krjwhoz8W00N8ItA",
"UCsd0xmaXyUGpD35IX4xrYEw",
"UCv3N90AXQxUtaSnKSqOoOpA",
"UCmTS2gGo4gjZLt8LxVnF3XA",
"UCTHAIlSOqquwOKnd_nCJgAQ",
"UCWI-ohtRu8eEeDj93hmUsUQ",
"UC50JofjMZpr3KCbOx6dHi_A",
"UCUOMtzQmEmhIAF8G_cha_Xw",
"UCdngmbVKX1Tgre699-XLlUA",
"UC5qEPWtHLFRIjhW_3xd5g2A",
"UC2UXDak6o7rBm23k3Vv5dww",
"UCyUBW72KU30dfAYWLVNZO8Q",
"UCVe8CMJF4caRzuckVYV8CaQ",
"UCvDqylEzCZJHNea6k1XMkQw",
"UCEDkO7wshcDZ7UZo17rPkzQ",
"UCt-Pwe2fODjH4Wuwf5VqE7A",
"UClLXKYEEM8OBBx85DOa6-cg",
"UCN0u98NPeC0AW3pD6MbVeNg",
"UCwAnu01qlnVg1Ai2AbtTMaA",
"UCthRsjtSd2ZTrfgjgnBXMPw",
"UCx3ZYdyVr8BUQDIhS1QggxA",
"UCo2avEF03abQmCXdU_eUGkg",
"UC4xKdmAXFh4ACyhpiQ_3qBw",
"UCIrOHHdl6C4IWv96N0EaHMA",
"UCJNUtbsYnt-8meFSj_yF50Q",
"UCEHFikgnRuLd1HYKTLrae9Q",
"UC1nBZreWJArdFgiIu3BTeGw",
"UC3kFmMxz3IdLdz4TxA9a4DQ",
"UCpa0VHHeSUV1D17X8MoX5xA"]

In [3]:
len(channel_ids)

50

In [4]:
# Get credentials and create an API client
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=api_key)

In [5]:
# channel request (getting all upload playlist ids from 50 selected tech career focused channels)

request = youtube.channels().list(
    part="snippet,contentDetails,statistics",
    id=','.join(channel_ids)
)
response = request.execute()
JSON(response)

<IPython.core.display.JSON object>

In [8]:
# get upload playlists

upload_ids = [item["contentDetails"]["relatedPlaylists"]["uploads"] for item in response["items"]]

In [10]:
video_ids = []
temp = []

In [12]:
for upload_playlist_id in upload_ids:
    count = 0
    #initial request
    request = youtube.playlistItems().list(
    part=['snippet'], 
    maxResults=50, 
    playlistId=upload_playlist_id)  #id=','.join(upload_ids))
    response = request.execute()

    #try to get all of the video id's from the response
    try:
        temp = [video["snippet"]["resourceId"]["videoId"] for video in response["items"]]
        video_ids = video_ids + temp
    except Exception as e:
        print(e)

    #increment page token
    next_page_token = response.get('nextPageToken')
    #print(next_page_token)
    #next_page_token
    
    # https://stackoverflow.com/questions/30653208/retrieve-video-ids-contained-in-a-playlist-youtube-api-v3
    # while there are still pages in the response to parse
    while next_page_token is not None: #and len(video_ids) < 100:
        count += 1
        #print(next_page_token)
        #make the same request, just get the next page of uploads (using nextPageToken which stores the token of the next page of search results
        request = youtube.playlistItems().list(
        part=['snippet'], 
        maxResults=50, 
        playlistId=upload_playlist_id,
        #id=','.join(upload_ids_test),
        pageToken = next_page_token)#id=','.join(upload_ids))

        response = request.execute()
        #make a temporary list of video ids on this page
        try:
            temp = [video["snippet"]["resourceId"]["videoId"] for video in response["items"]]
        except Exception as e:
            print(e)

        # add to video id list
        video_ids = video_ids + temp

        #increment page token
        next_page_token = response.get('nextPageToken')
        if count % 20 == 0:
            print(count)
        #print(next_page_token)

20
40
20
20
20
20
40
60
80
100
120
20
20


In [13]:
len(video_ids)

35299

In [14]:
def concatenate_df(response_json, output_df):    
    response_breakdown = {"snippet": ["channelTitle", "publishedAt", "channelId", "title", "categoryId", "description", "tags"], "contentDetails" : ["duration", "dimension", "definition", "caption", "licensedContent", "projection"], "statistics": ["viewCount", "likeCount", "favoriteCount", "commentCount"]}
    
    for video in response_json:

        row_to_add = {}
        
        #video id, if this isnt there we definitely need to just skip to the next, other parameters can be null
        try:
            row_to_add["id"] = video["id"]
        except KeyError:
            continue
        
        #for each category
        for category, category_list in response_breakdown.items():
            
            #for each feature
            for column_name in category_list:
                try:
                    row_to_add[column_name] = video[category][column_name]
                except KeyError:
                    row_to_add[column_name] = None
        
        row_to_add["Tag Count"] = len(row_to_add["tags"]) if row_to_add["tags"] != None else None
        row_to_add["Description Length (characters)"] = len(row_to_add["description"]) if row_to_add["description"] != None else None
        row_to_add["Description Length (words)"] = len(video["snippet"]["description"].split(" ")) if row_to_add["description"] != None else None
  
        output_df = pd.concat( [ output_df, pd.DataFrame.from_records([row_to_add]) ], ignore_index=True )
    return output_df

In [15]:
final_df = pd.DataFrame()

In [16]:
#https://blog.enterprisedna.co/how-to-write-a-list-to-csv-in-python/
#save video id's to file just in case
with open('video_ids.csv', 'w', newline='') as file:
    # Step 4: Using csv.writer to write the list to the CSV file
    writer = csv.writer(file)
    writer.writerow(video_ids) # Use writerow for single list

In [17]:
video_ids[0]

'2ruoagytC7w'

In [18]:
request = youtube.videos().list(
    part="snippet,contentDetails,statistics",
    id='2ruoagytC7w'
    )
    
response = request.execute()

JSON(response)

<IPython.core.display.JSON object>

In [21]:
#video statistics request

for i in range(0, len(video_ids), 50):
    
    # to avoid list index out of bounds error (dataset not divisible by 50)
    increment = i+50 if (i+50 < len(video_ids)-1) else len(video_ids)-1
    request = youtube.videos().list(
    part="snippet,contentDetails,statistics",
    id=','.join(video_ids[i:increment])
    )
    
    response = request.execute()


    final_df = concatenate_df(response["items"], final_df)
    
    if i % 1000 == 0:
        print(i)

final_df   

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000


,id,channelTitle,publishedAt,channelId,title,categoryId,description,tags,duration,dimension,...,caption,licensedContent,projection,viewCount,likeCount,favoriteCount,commentCount,Tag Count,Description Length (characters),Description Length (words)
0,2ruoagytC7w,PushToProd,2023-09-14T21:00:02Z,UCN0u98NPeC0AW3pD6MbVeNg,day in the life | Jr. Cybersecurity Engineer N...,28,Here’s the part 2 to my day in the life/ vlog!...,"[devops, azure, aws, cloud, docker, kubernetes...",PT7M22S,2d,...,false,True,rectangular,1032,61,0,22,39,162,20
1,gBH1LdSZyFU,PushToProd,2023-09-04T22:27:11Z,UCN0u98NPeC0AW3pD6MbVeNg,day in the life | Jr. Cybersecurity Engineer N...,28,Hey Guys! \n\nHope you enjoy Part 1 of this da...,"[devops, azure, aws, cloud, docker, kubernetes...",PT14M8S,2d,...,false,True,rectangular,2006,86,0,29,39,453,72
2,UhMJ0km4KLs,PushToProd,2023-08-15T21:30:03Z,UCN0u98NPeC0AW3pD6MbVeNg,Prodcast 6 | Is the end of tech near?,28,https://linktr.ee/pushtoprod\n\nHey push2prod ...,"[podcast, devops, azure, aws, cloud, docker, k...",PT19M29S,2d,...,false,True,rectangular,793,55,0,23,39,456,75
3,qYfxI07QrKo,PushToProd,2023-06-13T20:45:01Z,UCN0u98NPeC0AW3pD6MbVeNg,Prodcast 5 | Biology Degree to Data Scientist,28,"Hey push2prod squad. Welcome to back to the ""P...","[podcast, devops, azure, aws, cloud, docker, k...",PT25M9S,2d,...,false,True,rectangular,870,54,0,22,38,393,70
4,CKcYF3369LI,PushToProd,2023-06-07T14:00:10Z,UCN0u98NPeC0AW3pD6MbVeNg,From Suit & Tie to Bonnets and Durags: Embraci...,28,,"[podcast, devops, azure, aws, cloud, docker, k...",PT56S,2d,...,false,True,rectangular,1679,36,0,3,38,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35293,LMyFZVkJsLQ,Chris Sean,2017-01-14T05:43:23Z,UCu1xbgCV5o48h_BYCQD7KJg,All You NEED to BECOME A JUNIOR Web Developer ...,22,Learn to CODE in just a FEW months here: \n- M...,"[how to become a jr web developer, junior web ...",PT9M,2d,...,false,True,rectangular,192485,6246,0,504,18,336,36
35294,rG3HPCGB4hU,Chris Sean,2017-01-09T16:45:08Z,UCu1xbgCV5o48h_BYCQD7KJg,"At The Office, Bad Day & Knowing Your Value | ...",22,Learn to CODE in just a FEW months here: \n- M...,"[daily life of a developer, the day in the lif...",PT8M8S,2d,...,false,True,rectangular,6810,197,0,25,14,336,36
35295,HLgdtA-n_bo,Chris Sean,2016-12-30T00:38:56Z,UCu1xbgCV5o48h_BYCQD7KJg,What It Takes To Become A Web Developer in 2017,22,Learn to CODE in just a FEW months here: \n- M...,"[Learn code, html, php, css, jquery, javascrip...",PT7M35S,2d,...,false,True,rectangular,7054,247,0,43,20,336,36
35296,yAYJD4kKQ7c,Chris Sean,2016-12-22T17:12:12Z,UCu1xbgCV5o48h_BYCQD7KJg,Developers Q & A | Be Humble But Stay Competi...,22,Q & A | Be Humble But Stay Competitive \n\nNot...,"[developer q & a, how to become a developer, l...",PT14M,2d,...,false,True,rectangular,3807,134,0,16,14,385,38


In [23]:
final_df.to_csv('data/video_data_2.csv')  